# Preparations

In [74]:
pwd

/home/thomas/import/k8s-graph


In [75]:
ls 

ImageTragick.ipynb  neo4j                Neo4JHelm.ipynb
LICENSE             Neo4JHelm-4.4.ipynb  README.md


## Create namespace and set contect to work in this namespace

In [76]:
kubectl get namespace k8s-graph ||
kubectl create namespace k8s-graph

NAME        STATUS   AGE
k8s-graph   Active   26h


In [77]:
kubectl config set-context --current --namespace=k8s-graph

Context "minikube" modified.


should be empty

In [78]:
kubectl get all

NAME                                READY   STATUS      RESTARTS      AGE
pod/imagetragick-598dcdb6b6-j5ct6   1/1     Running     2 (49m ago)   26h
pod/my-neo4j-release-0              1/1     Running     0             12m
pod/neo4j-init-fhzlm                0/1     Completed   0             10m

NAME                             TYPE           CLUSTER-IP      EXTERNAL-IP    PORT(S)                                        AGE
service/imagetragick             LoadBalancer   10.101.174.5    10.101.174.5   8080:32338/TCP                                 26h
service/my-neo4j-release         LoadBalancer   10.109.43.96    10.109.43.96   7687:32273/TCP,7474:30668/TCP                  12m
service/my-neo4j-release-admin   ClusterIP      10.96.198.104   <none>         7687/TCP,7474/TCP                              12m
service/my-standalone-lb-neo4j   LoadBalancer   10.104.12.22    10.104.12.22   7474:31616/TCP,7473:31731/TCP,7687:32279/TCP   12m

NAME                           READY   UP-TO-DATE   

In [79]:
helm ls

NAME            	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART       	APP VERSION
my-neo4j-release	k8s-graph	1       	2024-04-05 19:33:20.556702152 +0200 CEST	deployed	neo4j-5.18.1	5.18.1     


In [80]:
ls neo4j/helm

values-4.4.yaml  values.yaml


In [81]:
cat neo4j/helm/values.yaml

neo4j:
  name: my-standalone
  resources:
    cpu: "4"
    memory: "4Gi"

  # Uncomment to set the initial password
  password: "hellohello"

  # Uncomment to use enterprise edition
  #edition: "enterprise"
  acceptLicenseAgreement: "yes"

volumes:
  data:
    mode: defaultStorageClass
    defaultStorageClass:
      requests:
        storage: 4Gi

image:
  customImage: neo4j:5.18.1

env:
  NEO4J_PLUGINS: '["apoc"]'


In [82]:
helm uninstall my-neo4j-release || true
kubectl delete pvc data-my-neo4j-release-0

release "my-neo4j-release" uninstalled
persistentvolumeclaim "data-my-neo4j-release-0" deleted


In [83]:
helm upgrade --install my-neo4j-release neo4j/neo4j  -f neo4j/helm/values.yaml


Release "my-neo4j-release" does not exist. Installing it now.
NAME: my-neo4j-release
LAST DEPLOYED: Fri Apr  5 19:45:51 2024
NAMESPACE: k8s-graph
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing neo4j.

Your release "my-neo4j-release" has been installed  in namespace "k8s-graph".

The neo4j user's password has been set to "hellohello".To view the progress of the rollout try:

  $ kubectl --namespace "k8s-graph" rollout status --watch --timeout=600s statefulset/my-neo4j-release

Once rollout is complete you can log in to Neo4j at "neo4j://my-neo4j-release.k8s-graph.svc.cluster.local:7687". Try:

  $ kubectl run --rm -it --namespace "k8s-graph" --image "neo4j:5.18.1" cypher-shell \
     -- cypher-shell -a "neo4j://my-neo4j-release.k8s-graph.svc.cluster.local:7687" -u neo4j -p "hellohello"

Graphs are everywhere!

Please consider using 'neo4j.passwordFromSecret' for improved security.


In [84]:
helm ls

NAME            	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART       	APP VERSION
my-neo4j-release	k8s-graph	1       	2024-04-05 19:45:51.974213293 +0200 CEST	deployed	neo4j-5.18.1	5.18.1     


In [85]:
kubectl get all

NAME                                READY   STATUS      RESTARTS      AGE
pod/imagetragick-598dcdb6b6-j5ct6   1/1     Running     2 (49m ago)   26h
pod/my-neo4j-release-0              0/1     Pending     0             0s
pod/neo4j-init-fhzlm                0/1     Completed   0             10m

NAME                             TYPE           CLUSTER-IP       EXTERNAL-IP    PORT(S)                                        AGE
service/imagetragick             LoadBalancer   10.101.174.5     10.101.174.5   8080:32338/TCP                                 26h
service/my-neo4j-release         ClusterIP      10.101.55.78     <none>         7687/TCP,7474/TCP                              0s
service/my-neo4j-release-admin   ClusterIP      10.100.130.201   <none>         7687/TCP,7474/TCP                              0s
service/my-standalone-lb-neo4j   LoadBalancer   10.106.23.82     <pending>      7474:30468/TCP,7473:32291/TCP,7687:30153/TCP   0s

NAME                           READY   UP-TO-DATE  

In [86]:
kubectl wait  --for=condition=Ready pod/my-neo4j-release-0

pod/my-neo4j-release-0 condition met


In [87]:
kubectl describe pod/my-neo4j-release-0

Name:             my-neo4j-release-0
Namespace:        k8s-graph
Priority:         0
Service Account:  default
Node:             minikube/192.168.49.2
Start Time:       Fri, 05 Apr 2024 19:45:53 +0200
Labels:           app=my-standalone
                  apps.kubernetes.io/pod-index=0
                  controller-revision-hash=my-neo4j-release-75fbfdf867
                  helm.neo4j.com/clustering=false
                  helm.neo4j.com/instance=my-neo4j-release
                  helm.neo4j.com/neo4j.loadbalancer=include
                  helm.neo4j.com/neo4j.name=my-standalone
                  helm.neo4j.com/pod_category=neo4j-instance
                  statefulset.kubernetes.io/pod-name=my-neo4j-release-0
Annotations:      checksum/my-neo4j-release-config: 5623926ee307148d81e0b7ced8dd54d1fe8f495b99dc71c27d35b2f622c27958
                  checksum/my-neo4j-release-env: 8994d0c0bddaf6a41d469e24298db773e03e0dce9ac8d7cf81aaa589cc68a764
Status:           Running
IP:               10.244.1

In [88]:
kubectl logs my-neo4j-release-0 

Installing Plugin 'apoc' from /var/lib/neo4j/labs/apoc-*-core.jar to /var/lib/neo4j/plugins/apoc.jar
Applying default values for plugin apoc to neo4j.conf
Changed password for user 'neo4j'. IMPORTANT: this change will only take effect if performed before the database is started for the first time.
2024-04-05 17:45:58.777+0000 INFO  Logging config in use: File '/config/user-logs.xml/user-logs.xml'
2024-04-05 17:45:58.791+0000 INFO  Command expansion is explicitly enabled for configuration
2024-04-05 17:45:58.803+0000 INFO  Starting...
2024-04-05 17:45:59.734+0000 INFO  This instance is ServerId{e3066285} (e3066285-8c8d-4769-8c8e-348e6c41dcf2)
2024-04-05 17:46:00.414+0000 INFO  ======== Neo4j 5.18.1 ========
2024-04-05 17:46:03.748+0000 INFO  Bolt enabled on 0.0.0.0:7687.


In [89]:
kubectl get pvc

NAME                      STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
data-my-neo4j-release-0   Bound    pvc-6f8a5cea-2497-4595-b21a-08b7b4b44fb8   4Gi        RWO            standard       12s


be sure to load apoc

In [90]:
kubectl exec -it my-neo4j-release-0 -- cat conf/neo4j.conf | grep apoc

#dbms.security.procedures.allowlist=apoc.coll.*,apoc.load.*,gds.*
dbms.security.procedures.unrestricted=apoc.*


## Deploy `Imagetragick`

We deploy a vulnerable version of `Imagemagick` using the Notebook [ImageTragick](ImageTragick.ipynb)

## Load cypher scripts

The cypher scripts need access to the API server

Start a proxy with `kubectl proxy --port=8001 --address=172.26.0.1 --accept-hosts='^*$'` in a different terminal.

If you use a different setup for the API_SERVER, change the variable in the next line

In [91]:
export API_SERVER=http://172.26.0.1:8001
# curl $API_SERVER

## Create a vulnerable rolebinding to the cluster

This sets up the scene for this [attack to take over the cluster by uploading images to the vulnerable ImageTragick](https://github.com/thomasfricke/training-kubernetes-security/blob/main/OpenShift/Overview.ipynb)

In [92]:
kubectl get ClusterRoleBinding cluster-admin-binding || \
kubectl create -f - <<EOF
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: cluster-admin-binding
subjects:
  - kind: ServiceAccount
    name: default 
    namespace: k8s-graph
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
EOF

NAME                    ROLE                        AGE
cluster-admin-binding   ClusterRole/cluster-admin   26m


In [93]:
kubectl get clusterrolebinding cluster-admin-binding -o yaml

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  creationTimestamp: "2024-04-05T17:20:03Z"
  name: cluster-admin-binding
  resourceVersion: "1186092"
  uid: e6c52b1f-b5c1-4109-8298-c7d28d3e3ddb
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: ServiceAccount
  name: default
  namespace: k8s-graph


### Load all `*.cypher` scripts and the `import.sh` script into a config map

In [94]:
ls neo4j/import -R

cypher

neo4j/import/cypher:
clusterrolebindings.cypher  import.sh            serviceaccounts.cypher
clusterroles.cypher         pods.cypher          services.cypher
create-cve.cypher           rolebindings.cypher  
erase.cypher                roles.cypher         


In [95]:
kubectl delete configmap cyphers || true
kubectl create configmap cyphers $(for i in neo4j/import/cypher/*{cypher,sh}; do echo -n "--from-file=$i "  ; done) 

configmap "cyphers" deleted
configmap/cyphers created


In [96]:
kubectl get configmap cyphers -o yaml

apiVersion: v1
data:
  clusterrolebindings.cypher: "//load clusterrolebindings\n\nCALL apoc.load.json(\"${API_SERVER}/apis/rbac.authorization.k8s.io/v1/clusterrolebindings\")\nYIELD
    value as clusterrolebindinglist\n\nWITH clusterrolebindinglist\nUNWIND clusterrolebindinglist.items
    AS clusterrolebinding\n  MERGE (crb:ClusterRoleBinding {name: clusterrolebinding.metadata.name,\n
    \               kind: \"ClusterRoleBinding\" } )\n  WITH clusterrolebinding \n
    \ UNWIND clusterrolebinding.subjects as subject\n  MATCH (s:ServiceAccount {name:
    subject.name,\n                           namespace: subject.namespace})\n  MATCH
    (cr:ClusterRole {\n                                      kind: clusterrolebinding.roleRef.kind,\n
    \                                     name: clusterrolebinding.roleRef.name\n
    \                    }) \n  MERGE (s)-[:ClusterRoleBinding  {name: clusterrolebinding.metadata.name,\n
    \                                     kind: \"ClusterRoleBindi

## Username and Password

1. are in the secret
2. gets exported to environment vars

In [97]:
kubectl get secrets

NAME                                     TYPE                 DATA   AGE
my-standalone-auth                       Opaque               1      15s
sh.helm.release.v1.my-neo4j-release.v1   helm.sh/release.v1   1      15s


In [98]:
kubectl get secrets my-standalone-auth -o yaml

apiVersion: v1
data:
  NEO4J_AUTH: bmVvNGovaGVsbG9oZWxsbw==
kind: Secret
metadata:
  annotations:
    meta.helm.sh/release-name: my-neo4j-release
    meta.helm.sh/release-namespace: k8s-graph
  creationTimestamp: "2024-04-05T17:45:52Z"
  labels:
    app: my-standalone
    app.kubernetes.io/managed-by: Helm
  name: my-standalone-auth
  namespace: k8s-graph
  resourceVersion: "1187965"
  uid: 1af8e603-a140-4351-88b2-8d6f4a721c49
type: Opaque


In [99]:
NEO4J_AUTH=$(kubectl get secrets my-standalone-auth -o jsonpath='{.data.NEO4J_AUTH}' | base64 --decode)
NEO4J_USERNAME=$(echo $NEO4J_AUTH | cut -f1 -d/)
NEO4J_PASSWORD=$(echo $NEO4J_AUTH | cut -f2 -d/)
echo $NEO4J_AUTH $NEO4J_USERNAME $NEO4J_PASSWORD


neo4j/hellohello neo4j hellohello


In [100]:
kubectl delete job neo4j-init || true

kubectl create -f - <<EOF
apiVersion: batch/v1
kind: Job
metadata:
  name: neo4j-init
  labels:
    app: neo4j-init
spec:
  template:
    metadata:
      labels:
        app: neo4j-init
    spec:
      containers:
      - name: neo4j-init
        image: neo4j:5.18.1
        args:
          - "/import/import.sh"
        env:
        - name: API_SERVER
          value: ${API_SERVER}
        - name: NEO4J_PASSWORD
          value: ${NEO4J_PASSWORD}
        - name: NEO4J_USERNAME
          value: ${NEO4J_USERNAME}
        - name: NEO4J_ADDRESS
          value: 'neo4j://my-neo4j-release:7687'
        volumeMounts:
        - name: cyphers
          mountPath: /import
      restartPolicy: Never
      volumes:
        - name: cyphers
          configMap:
            name: cyphers
            defaultMode: 0777
EOF

job.batch "neo4j-init" deleted
job.batch/neo4j-init created


In [101]:
kubectl wait  --for=condition=complete job/neo4j-init

job.batch/neo4j-init condition met


In [102]:
kubectl logs -l app=neo4j-init 

importing erase ...done
importing serviceaccounts ...done
importing pods ...done
importing services ...done
importing create-cve ...done
importing roles ...done
importing clusterroles ...done
importing clusterrolebindings ...done
importing rolebindings ...done


In [103]:
kubectl get service my-neo4j-release -o jsonpath='{.spec.type}'

ClusterIP


In [104]:
kubectl patch svc my-neo4j-release --patch='{"spec":{"type":"LoadBalancer"}}'

service/my-neo4j-release patched


In [105]:
kubectl get svc my-neo4j-release -o jsonpath='

Open a browser

http://{.spec.clusterIP}:7474/browser

and login as 

user: neo4j
password: '$NEO4J_PASSWORD'

'



Open a browser

http://10.101.55.78:7474/browser

and login as 

user: neo4j
password: hellohello



# Commands to be run in the Neo4 browser

## Increase number of items

## Useful commands

### Show all items

```
MATCH (g)
RETURN g
```

### Show *ALL* items connected to a CVE

```
MATCH g= (c:CVE)-[*1..7]-(connected)
RETURN g
```

### Show all  `Services` connected to a CVE

```
MATCH g= (c:CVE)-[*1..7]-(s:Service)
RETURN g
```

### Show all  `ServiceAccounts` connected to a CVE

```
MATCH g= (c:CVE)-[*1..7]-(s:ServiceAccount)
RETURN g
```

### Show all  `ClusterRoles` connected to a CVE

```
MATCH g= (c:CVE)-[*1..7]-(cr:ClusterRole)
RETURN g
```




In [106]:
# kubectl delete namespace k8s-graph